In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reviews/reviews1.csv
/kaggle/input/reviews/reviews4.csv
/kaggle/input/reviews/reviews3.csv


In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
import keras.utils as ku
import string

Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

In [4]:
curr_dir = '/kaggle/input/reviews/'
reviews = []
for file in os.listdir(curr_dir):
    review9_df = pd.read_csv(curr_dir + file)
    reviews.extend(list(review9_df.body.values))

len(reviews)

30

In [5]:
def clean(text):
    text = "".join(k for k in text if k not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii",'ignore')
    return text 

text = [clean(x) for x in reviews]
text[:10]

['just got mine last week for 33k and now they are launching one plus 7 for 33k which has d latest processor better screen speaker and finger print sensor amazon has been really unfair by selling 6t at 33k please dont go for 6t they just want to clear the stocks wait for a while you will get this more a more cheaper price highly dissappointed',
 'the phone got stuck while using one of camera features its not even getting restarted the screen is freezed thus not allowing to proceed further hope i get a replacement soon',
 'build quality is good phone is looking handsome two things bothering me one is notification light there is no notification light on top of screen so i need to awake my phone everytime to check any message or so second thing is fingerprint sensor it is very sensible i must say unlike the back one i need to put my finger in proper way otherwise i wont be able to unlock my phone',
 'damaged',
 'i am a big fan of apple have been using iphone from last 5 years wanted to ch

In [6]:
tokenizer = Tokenizer()

def get_tokens(corpus):
    tokenizer.fit_on_texts(text)
    total_no_words = len(tokenizer.word_index) + 1
    input = []
    for line in text:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram = token_list[:i+1]
            input.append(n_gram)
    return input, total_no_words

input, total_no_words = get_tokens(text)
input[:10]

[[64, 149],
 [64, 149, 464],
 [64, 149, 464, 101],
 [64, 149, 464, 101, 294],
 [64, 149, 464, 101, 294, 10],
 [64, 149, 464, 101, 294, 10, 86],
 [64, 149, 464, 101, 294, 10, 86, 3],
 [64, 149, 464, 101, 294, 10, 86, 3, 77],
 [64, 149, 464, 101, 294, 10, 86, 3, 77, 87],
 [64, 149, 464, 101, 294, 10, 86, 3, 77, 87, 20]]

In [7]:
def padded_sequences(input):
    max_len = max([len(x) for x in input])
    input = np.array(pad_sequences(input, maxlen=max_len, padding='pre'))
    
    predictors, label = input[:,:-1],input[:,-1]
    label = ku.to_categorical(label, num_classes=total_no_words)
    return predictors, label, max_len

predictors, label, max_sequence_len = padded_sequences(input)
print(len(predictors))
print(total_no_words)

3913
1151


In [8]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(100,return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_no_words)

In [ ]:
model.fit(predictors, label, epochs=10,batch_size=200)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
3913/3913 [==============================] - 91s 23ms/step - loss: 7.0428
Epoch 2/100
 500/3913 [==>...........................] - ETA: 1:10 - loss: 7.0176

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print (generate_text("two days back i", 15, model, max_sequence_len))

In [ ]:
print (generate_text("i got this phone", 15, model, max_sequence_len))